In [181]:
import requests
import argparse
import time
import json
from io import StringIO
import gzip
import csv
import codecs
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import sys
#reload(sys)
#sys.setdefaultencoding('utf8')

# parse the command line arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-d","--domain",required=True,help="The domain to target ie. cnn.com")
#args = vars(ap.parse_args())

#domain = args['domain']
domain= "usga.org"


# list of available indices
#index_list = ["2019-04","2019-09","2019-13"]
index_list = ["2019-13","2019-09","2019-04"]

#
# Searches the Common Crawl Index for a domain.
#
def search_domain(domain):

    record_list = []
    
    print ("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print ("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print ("[*] Added %d results." % len(records))
            
    
    print ("[*] Found a total of %d hits." % len(record_list))
    
    return record_list        

#
# Downloads a page from Common Crawl - adapted graciously from @Smerity - thanks man!
# https://gist.github.com/Smerity/56bc6f21a8adec920ebf
#
def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://aws-publicdatasets.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = StringIO.StringIO(resp.content)
    f = raw_data
    
    # What we have now is just the WARC response, formatted:
    data = f.read()
    
    response = ""
    
    if len(data):
        try:
            warc, header, response = data.strip().split('\r\n\r\n', 2)
        except:
            pass
            
    return response

#
# Extract links from the HTML  
#
def extract_external_links(html_content,link_list):

    parser = BeautifulSoup(html_content)
        
    links = parser.find_all("a")
    
    if links:
        
        for link in links:
            href = link.attrs.get("href")
            
            if href is not None:
                
                if domain not in href:
                    if href not in link_list and href.startswith("http"):
                        print ("[*] Discovered external link: %s" % href)
                        link_list.append(href)

    return link_list





#link_list   = []
#print(record_list)
#c=0


'''
for record in record_list:
	
    
    html_content = download_page(record)
    
    print "[*] Retrieved %d bytes for %s" % (len(html_content),record['url'])
    
    link_list = extract_external_links(html_content,link_list)
    

print "[*] Total external links discovered: %d" % len(link_list)

with codecs.open("%s-links.csv" % domain,"wb",encoding="utf-8") as output:

    fields = ["URL"]
    
    logger = csv.DictWriter(output,fieldnames=fields)
    logger.writeheader()
    
    for link in link_list:
        logger.writerow({"URL":link})
'''

'\nfor record in record_list:\n\t\n    \n    html_content = download_page(record)\n    \n    print "[*] Retrieved %d bytes for %s" % (len(html_content),record[\'url\'])\n    \n    link_list = extract_external_links(html_content,link_list)\n    \n\nprint "[*] Total external links discovered: %d" % len(link_list)\n\nwith codecs.open("%s-links.csv" % domain,"wb",encoding="utf-8") as output:\n\n    fields = ["URL"]\n    \n    logger = csv.DictWriter(output,fieldnames=fields)\n    logger.writeheader()\n    \n    for link in link_list:\n        logger.writerow({"URL":link})\n'

In [182]:
record_list = search_domain(domain)

[*] Trying target domain: usga.org
[*] Trying index 2019-13
[*] Added 12497 results.
[*] Trying index 2019-09
[*] Added 10591 results.
[*] Trying index 2019-04
[*] Added 4223 results.
[*] Found a total of 27311 hits.


In [183]:
unique_url = set()
for record in record_list:
    if(len(unique_url)==100):
        break
    if("golf" in record["url"]):
        unique_url.add(record["url"])
    

In [184]:
unique_url

{'http://www.usga.org/articles/2010/01/the-origin-of-pebble-beach-golf-links-54931.html',
 'http://www.usga.org/articles/2010/04/us-amateur-in-1929-at-pebble-beach-led-bob-jones-to-mackenzie-and-eventually-to-design-of-augusta-national-golf-club-2147486003.html',
 'http://www.usga.org/articles/2011/02/course-care-an-appeal-for-return-of-golf-course-etiquette-2147496028.html',
 'http://www.usga.org/articles/2011/08/part-i-golf-and-the-military-21474842287.html',
 'http://www.usga.org/articles/2016/09/2017-golf-innovation-symposium-to-vancouver.html',
 'http://www.usga.org/articles/2018/01/usga-forms-new-alliance-with-regional-golf-associations.html',
 'http://www.usga.org/articles/2018/07/repairing-ball-marks--5-things-every-golfer-should-know-.html',
 'http://www.usga.org/articles/2018/07/usga--the-r-a-green-reading-materials-rules-of-golf.html',
 'http://www.usga.org/articles/2018/08/braden-thornberry-golf-worlds-top-amateur-player-mccormack-medal.html',
 'http://www.usga.org/champion

In [185]:
n=0
notfound=0
for url in unique_url:
    try:
        page=urlopen(url)
    except:
        notfound = notfound +1
        n = n-1
        continue
    soup = BeautifulSoup(page,'html.parser')
    all_data = soup.find_all("p")
    filename='golf_cc'+str(n)+'.txt'
    f=open(filename,'w')
    for j in range(len(all_data)):
        #if(j<3 or j >= len(all_data)-4):
            #continue
        f.write(all_data[j].get_text()+"\n")
    f.close()
            #print(url)
    n+=1

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [186]:
notfound

0